In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import math
import time
import re
from copy import deepcopy
from functools import lru_cache
from concurrent.futures import ProcessPoolExecutor

In [ ]:
def read_data_file(filename):
    # Dicionários para armazenar os dados
    vehicles = []
    customers = []
    
    with open(filename, 'r') as file:
        lines = file.readlines()
        
        # Flags para identificar as seções do arquivo
        vehicle_section = False
        customer_section = False
        
        for line in lines:
            # Remover espaços em branco no início e fim
            line = line.strip()
            
            # Verificar se é a seção de veículos
            if line.startswith("VEHICLE"):
                vehicle_section = True
                customer_section = False
                continue
            # Verificar se é a seção de clientes
            elif line.startswith("CUSTOMER"):
                vehicle_section = False
                customer_section = True
                # Pular o cabeçalho da tabela de clientes
                next_line = True
                continue
            
            # Processar seção de veículos
            if vehicle_section and line:
                # Usar expressão regular para extrair números
                numbers = re.findall(r'\d+', line)
                if len(numbers) >= 2:
                    vehicles.append({
                        'number': int(numbers[0]),
                        'capacity': int(numbers[1])
                    })
            
            # Processar seção de clientes
            if customer_section and line:
                # Pular linhas de cabeçalho
                if any(word in line.lower() for word in ['cust', 'no.', 'xcoord', 'ydcoord']):
                    continue
                
                # Extrair todos os números da linha
                numbers = re.findall(r'-?\d+', line)
                if len(numbers) >= 7:
                    customers.append({
                        'cust_no': int(numbers[0]),
                        'xcoord': int(numbers[1]),
                        'ycoord': int(numbers[2]),
                        'demand': int(numbers[3]),
                        'ready_time': int(numbers[4]),
                        'due_date': int(numbers[5]),
                        'service_time': int(numbers[6])
                    })
    
    return vehicles, customers

if __name__ == "__main__":
    filename = "R111.txt"
    vehicles, customers = read_data_file(filename)
    
    # Exibir informações lidas
    print("Veículos:")
    for vehicle in vehicles:
        print(f"Número: {vehicle['number']}, Capacidade: {vehicle['capacity']}")
    
    print("\nClientes (mostrando os 5 primeiros):")
    for customer in customers[:5]:
        print(f"Cliente {customer['cust_no']}:")
        print(f"  Coordenadas: ({customer['xcoord']}, {customer['ycoord']})")
        print(f"  Demanda: {customer['demand']}")
        print(f"  Janela de tempo: [{customer['ready_time']}, {customer['due_date']}]")
        print(f"  Tempo de serviço: {customer['service_time']}")

Veículos:
Número: 25, Capacidade: 200

Clientes (mostrando os 5 primeiros):
Cliente 0:
  Coordenadas: (35, 35)
  Demanda: 0
  Janela de tempo: [0, 230]
  Tempo de serviço: 0
Cliente 1:
  Coordenadas: (41, 49)
  Demanda: 10
  Janela de tempo: [15, 204]
  Tempo de serviço: 10
Cliente 2:
  Coordenadas: (35, 17)
  Demanda: 7
  Janela de tempo: [18, 202]
  Tempo de serviço: 10
Cliente 3:
  Coordenadas: (55, 45)
  Demanda: 13
  Janela de tempo: [54, 187]
  Tempo de serviço: 10
Cliente 4:
  Coordenadas: (55, 20)
  Demanda: 19
  Janela de tempo: [138, 169]
  Tempo de serviço: 10


In [11]:
# Estruturas de dados já implementadas na parte anterior
vehicles, customers = read_data_file("R111.txt")

# 1. Implementação da Heurística de Solomon
class SolomonVRPTW:
    def __init__(self, vehicles, customers):
        self.vehicles = vehicles
        self.customers = customers
        self.depot = customers[0]
        self.max_vehicles = vehicles[0]['number']
        self.vehicle_capacity = vehicles[0]['capacity']
        
    def calculate_distance(self, c1, c2):
        return math.sqrt((c1['xcoord'] - c2['xcoord'])**2 + (c1['ycoord'] - c2['ycoord'])**2)
    
    def initialize_problem(self):
        # Pré-calcular todas as distâncias
        self.distances = [[0]*len(self.customers) for _ in range(len(self.customers))]
        for i in range(len(self.customers)):
            for j in range(len(self.customers)):
                self.distances[i][j] = self.calculate_distance(self.customers[i], self.customers[j])
        
        # Lista de clientes não atendidos (excluindo o depósito)
        self.unserved = self.customers[1:]
        
        # Rotas iniciais vazias
        self.routes = []
    
    def compute_insertion_cost(self, route, customer, position):
        if position == 0:
            prev_cust = self.depot
            next_cust = route[0]
        elif position == len(route):
            prev_cust = route[-1]
            next_cust = self.depot
        else:
            prev_cust = route[position-1]
            next_cust = route[position]
        
        # Custo da inserção
        added_distance = (self.distances[prev_cust['cust_no']][customer['cust_no']] + 
                         self.distances[customer['cust_no']][next_cust['cust_no']] - 
                         self.distances[prev_cust['cust_no']][next_cust['cust_no']])
        
        return added_distance
    
    def check_time_window(self, route, customer, position):
        # Cria uma rota temporária para verificar a viabilidade
        temp_route = deepcopy(route)
        temp_route.insert(position, customer)
        
        current_time = 0
        prev_cust = self.depot
        
        for cust in temp_route:
            # Tempo de viagem até o próximo cliente
            travel_time = self.distances[prev_cust['cust_no']][cust['cust_no']]
            arrival_time = current_time + travel_time
            
            # Verificar janela de tempo
            if arrival_time < cust['ready_time']:
                current_time = cust['ready_time']
            elif arrival_time > cust['due_date']:
                return False
            
            # Tempo de serviço
            current_time += cust['service_time']
            prev_cust = cust
        
        # Volta ao depósito
        travel_time = self.distances[prev_cust['cust_no']][self.depot['cust_no']]
        if current_time + travel_time > self.depot['due_date']:
            return False
        
        return True
    
    def check_capacity(self, route, customer):
        total_demand = sum(c['demand'] for c in route) + customer['demand']
        return total_demand <= self.vehicle_capacity
    
    def find_best_insertion(self, customer):
        best_cost = float('inf')
        best_route_idx = -1
        best_position = -1
        
        for route_idx, route in enumerate(self.routes):
            for position in range(len(route)+1):
                if (self.check_time_window(route, customer, position) and 
                    self.check_capacity(route, customer)):
                    
                    cost = self.compute_insertion_cost(route, customer, position)
                    if cost < best_cost:
                        best_cost = cost
                        best_route_idx = route_idx
                        best_position = position
        
        # Se não encontrou inserção nas rotas existentes, tentar nova rota
        if best_route_idx == -1 and len(self.routes) < self.max_vehicles:
            if self.check_time_window([], customer, 0):
                best_route_idx = len(self.routes)
                best_position = 0
                best_cost = (self.distances[self.depot['cust_no']][customer['cust_no']] + 
                              self.distances[customer['cust_no']][self.depot['cust_no']])
        
        return best_route_idx, best_position, best_cost
    
    def construct_solution(self, mi=1, lambda_=1):
        # Ordenar clientes por janela de tempo mais cedo
        self.unserved.sort(key=lambda x: x['due_date'])
        
        while self.unserved:
            # Selecionar o próximo cliente
            customer = self.unserved.pop(0)
            
            # Encontrar a melhor inserção
            route_idx, position, _ = self.find_best_insertion(customer)
            
            if route_idx != -1:
                if route_idx == len(self.routes):
                    # Nova rota
                    self.routes.append([customer])
                else:
                    # Inserir na rota existente
                    self.routes[route_idx].insert(position, customer)
            else:
                # Não foi possível inserir - deixar não atendido
                pass
    
    def calculate_total_distance(self):
        total = 0
        for route in self.routes:
            if not route:
                continue
                
            # Do depósito ao primeiro cliente
            total += self.distances[self.depot['cust_no']][route[0]['cust_no']]
            
            # Entre clientes
            for i in range(len(route)-1):
                total += self.distances[route[i]['cust_no']][route[i+1]['cust_no']]
            
            # Do último cliente ao depósito
            total += self.distances[route[-1]['cust_no']][self.depot['cust_no']]
        
        return total
    
    def get_unserved_customers(self):
        served = set()
        for route in self.routes:
            for cust in route:
                served.add(cust['cust_no'])
        
        return [c for c in self.customers[1:] if c['cust_no'] not in served]

# 2. Resolver o caso descrito no arquivo
def solve_problem():
    solver = SolomonVRPTW(vehicles, customers)
    solver.initialize_problem()
    solver.construct_solution()
    
    print("\nSolução Inicial:")
    print(f"Número de rotas: {len(solver.routes)}")
    print(f"Distância total: {solver.calculate_total_distance()}")
    print(f"Clientes não atendidos: {len(solver.get_unserved_customers())}")
    
    for i, route in enumerate(solver.routes):
        print(f"\nRota {i+1}:")
        print("Depósito -> ", end="")
        print(" -> ".join(str(c['cust_no']) for c in route), end="")
        print(" -> Depósito")
        print(f"Demanda total: {sum(c['demand'] for c in route)}")
    
    return solver

# 3. Introduzir aleatoriedades e rodar 1000 vezes
def run_multiple_times():
    best_solution = None
    best_distance = float('inf')
    best_unserved = float('inf')
    
    for _ in range(1000):
        mi = random.uniform(0, 2)
        lambda_ = random.uniform(0, 2)
        
        solver = SolomonVRPTW(vehicles, customers)
        solver.initialize_problem()
        solver.construct_solution(mi, lambda_)
        
        current_distance = solver.calculate_total_distance()
        current_unserved = len(solver.get_unserved_customers())
        
        # Critério: minimizar não atendidos primeiro, depois distância
        if (current_unserved < best_unserved or 
            (current_unserved == best_unserved and current_distance < best_distance)):
            best_solution = deepcopy(solver)
            best_distance = current_distance
            best_unserved = current_unserved
    
    print("\nMelhor Solução após 1000 execuções:")
    print(f"Parâmetros (mi, lambda): {mi}, {lambda_}")
    print(f"Número de rotas: {len(best_solution.routes)}")
    print(f"Distância total: {best_distance}")
    print(f"Clientes não atendidos: {best_unserved}")
    
    for i, route in enumerate(best_solution.routes):
        print(f"\nRota {i+1}:")
        print("Depósito -> ", end="")
        print(" -> ".join(str(c['cust_no']) for c in route), end="")
        print(" -> Depósito")
        print(f"Demanda total: {sum(c['demand'] for c in route)}")

if __name__ == "__main__":
    # Resolver o problema uma vez
    solver = solve_problem()
    
    # Rodar múltiplas vezes com aleatoriedade
    run_multiple_times()


Solução Inicial:
Número de rotas: 8
Distância total: 1619.4007433998786
Clientes não atendidos: 0

Rota 1:
Depósito -> 92 -> 45 -> 36 -> 62 -> 69 -> 27 -> 33 -> 76 -> 79 -> 78 -> 9 -> 81 -> 66 -> 51 -> 70 -> Depósito
Demanda total: 196

Rota 2:
Depósito -> 75 -> 42 -> 15 -> 23 -> 67 -> 39 -> 40 -> 73 -> 22 -> 41 -> 56 -> 43 -> 57 -> 13 -> Depósito
Demanda total: 200

Rota 3:
Depósito -> 28 -> 29 -> 65 -> 71 -> 30 -> 90 -> 64 -> 11 -> 88 -> 8 -> 84 -> 17 -> 85 -> 91 -> 37 -> 97 -> Depósito
Demanda total: 194

Rota 4:
Depósito -> 14 -> 61 -> 16 -> 86 -> 44 -> 38 -> 87 -> 53 -> 55 -> 4 -> 74 -> 25 -> 24 -> Depósito
Demanda total: 199

Rota 5:
Depósito -> 21 -> 12 -> 52 -> 83 -> 82 -> 48 -> 32 -> 54 -> 6 -> 99 -> 94 -> 100 -> Depósito
Demanda total: 199

Rota 6:
Depósito -> 18 -> 7 -> 46 -> 47 -> 49 -> 19 -> 63 -> 10 -> 20 -> 35 -> 34 -> 3 -> 68 -> Depósito
Demanda total: 198

Rota 7:
Depósito -> 59 -> 96 -> 31 -> 1 -> 50 -> 80 -> 72 -> 2 -> 98 -> 93 -> 5 -> 60 -> Depósito
Demanda total: 